In [49]:
import netCDF4 as nc
import numpy as np
def readCM1(fname):
    with nc.Dataset(fname) as f:
        dbz=f['dbz'][:,:,:,:]
        dbz=np.ma.array(dbz,mask=dbz<10)
        qr=f['qr'][:,:,:,:]
        qs=f['qs'][:,:,:,:]
        qg=f['qg'][:,:,:,:]
        qc=f['qc'][:,:,:,:]
        qv=f['qv'][:,:,:,:]
        T=f['th'][:,:,:,:]
        press=f['prs'][:,:,:,:]
        ncr=f['ncr'][:,:,:,:]
        ncs=f['ncs'][:,:,:,:]
        ncg=f['ncg'][:,:,:,:]
    return dbz,qr,qs,qg,qc,qv,T,press,ncr,ncs,ncg

def readObs(fname):
    with nc.Dataset(fname) as f:
        zKa=f['zKu'][:,:,:]
        zKu=f['zKa'][:,:,:]
        piaKu=f['piaKu'][:,:]
        piaKa=f['piaKa'][:,:]
    return zKa,zKu,piaKu,piaKa
ntot=0
x1_L=[]
x2_L=[]
y_L=[]
for it in range(20,34):
    fname1="cm1out/cm1out_0000%2.2i.nc"%it
    fname2="cm1out/simObs_%4.4i.nc"%it
    dbz,qr,qs,qg,qc,qv,T,press,ncr,ncs,ncg=readCM1(fname1)
    rho=press/(287*T)
    zKa,zKu,piaKu,piaKa=readObs(fname2)
    a=np.nonzero(piaKu>1)
    print(len(a[0]))
    ntot+=len(a[0])
    nz,ny,nx=zKa.shape
    for i1,j1 in zip(a[0],a[1]):
        if i1>2 and i1<ny-3 and j1>2 and j1<nx-3:
            zKu1=np.log10((10**(0.1*zKu[:,i1-2:i1+3,j1-2:j1+3])).mean(axis=(1,2))+1e-9)*10
            zKa1=np.log10((10**(0.1*zKa[:,i1-2:i1+3,j1-2:j1+3])).mean(axis=(1,2))+1e-9)*10
            piaKa1=np.log10((10**(-0.1*piaKa[i1-2:i1+3,j1-2:j1+3])).mean())*(-10)
            piaKu1=np.log10((10**(-0.1*piaKu[i1-2:i1+3,j1-2:j1+3])).mean())*(-10)
            x1_L.append(np.array([zKu1[::-1],zKa1[::-1]]).T)
            x2_L.append([piaKu1,piaKa1])
            x=np.concatenate((zKu1[::-1],zKa1[::-1],[piaKu1],[piaKa1]))
            qtot=(qr[0,:,i1-2:i1+3,j1-2:j1+3]+qs[0,:,i1-2:i1+3,j1-2:j1+3]+qg[0,:,i1-2:i1+3,j1-2:j1+3]).mean(axis=(1,2))
            pwc_tot=qtot*rho[0,:,i1,j1]*1e3
            y_L.append(pwc_tot)
            #break
    #break

800
860
926
998
1090
1230
1292
1357
1452
1477
1580
1676
1722
1808


In [50]:
print(np.array(x1_L).shape)
print(np.array(x2_L).shape)
print(piaKu1,piaKa1)
import xarray as xr
x1=xr.DataArray(np.array(x1_L),dims=['ntot','nlev','nband'])
x2=xr.DataArray(np.array(x2_L),dims=['ntot','nband'])
y=xr.DataArray(np.array(y_L),dims=['ntot','nlev'])
ds=xr.Dataset({'x1':x1,'x2':x2,'y':y})
ds.to_netcdf('training_data_CM1.nc')

(16741, 80, 2)
(16741, 2)
1.287805715335658 6.9345390629614725
